# SQL: Basic queries on relational databases

## Relational databases: Review

A  **relational database** is a way of structuring information in tables, rows, and columns. You could compare it to an excel file (database) with several sheets (tables) that contain tabular data (rows and columns). The particularity of this database is its ability to create relationships between those tables by joining them.

What is exactly a `join` ? Let's take an example by looking to the schema of a database of a music producer. You can see here that certain fields are common between several tables.

For example: `ArtistID` exists in `artists` and in `albums`. Consequently the information from an artist can be included (_joined_) to the information of his/her albums.

<img src="https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

SQL (Structured Query Language) is one of the most common languages for interacting with a relational database. It allows you to perform efficient and precise queries on the database, including queries among different tables.

SQL mainly allows you to `select`, `insert`, `update` or `delete` data from the DB.

Before diving in the exercises:

- You can learn the basics of SQL language by practicing [this tutorial](https://www.w3schools.com/sql/sql_syntax.asp)
- Try to understand the structure of the DB based on its schema. You will get more information about it [here](https://www.sqlitetutorial.net/sqlite-sample-database/)

Now, let's do some exercices by using SQLite and the python module `sqlite3`. You don't need to install it, it is included in the Python distribution. Read more: [SQLite3 Documentation](https://docs.python.org/3/library/sqlite3.html).




### Connetion to the database

We need to create a new database and open a database connection to allow `sqlite3` to work with it. Call `sqlite3.connect()` to create a connection to the database in the current working directory, implicitly creating it if it does not exist.

In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor. Call `con.cursor()` to create the `Cursor`.

In [11]:
import sqlite3

connexion = sqlite3.connect("../data/chinook.db")
cursor = connexion.cursor()

In [25]:
import sqlite3

# Replace 'your_database.db' with the path to your .db file
db_file = "../data/chinook.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute an SQL query to retrieve the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Close the database connection
# conn.close()

# Print the list of tables
for table in tables:
    print(table[0])


albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1


## Basic Queries

**1. Show the 10 first entries of the table `artists`**

The basic command in SQL is `SELECT` which is used for retrieving elements of certain columns `FROM` a table. `SELECT` indicates the columns to retrieve. By using `*` we chose to get keep all the columns. The keyword `LIMIT` is used for retrieving only the first `n` rows from the table.

You can use `cursor.execute()` to execute that query and call `cursor.fetchall()` to fetch the resulting rows.


In [26]:
cursor.execute("""
    SELECT
        *
    FROM
        artists
    LIMIT 10;
""")
cursor.fetchall()

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains'),
 (6, 'Antônio Carlos Jobim'),
 (7, 'Apocalyptica'),
 (8, 'Audioslave'),
 (9, 'BackBeat'),
 (10, 'Billy Cobham')]

In [27]:
# EXPORT DATA SCHEMA 
import sqlite3

db_file = '../data/chinook.db'
output_file = 'schema.txt'  # The file where schema information will be saved

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute an SQL query to retrieve the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Open the output file in write mode
with open(output_file, 'w') as f:
    # Iterate through each table and write the schema to the file
    for table in tables:
        table_name = table[0]
        f.write(f"Table Name: {table_name}\n")
        
        # Use PRAGMA statement to get table info
        cursor.execute(f"PRAGMA table_info({table_name});")
        table_info = cursor.fetchall()
        
        # Write the schema information for each column to the file
        for column in table_info:
            column_name = column[1]
            data_type = column[2]
            not_null = "NOT NULL" if column[3] else ""
            default_value = f"DEFAULT {column[4]}" if column[4] is not None else ""
            
            f.write(f"  - {column_name} {data_type} {not_null} {default_value}\n")
        
        f.write("\n" + "="*40 + "\n")

print(f"Schema information has been saved to '{output_file}'.")


Schema information has been saved to 'schema.txt'.


In [29]:
# View all TABLES 
import sqlite3
import pandas as pd

# Replace 'your_database.db' with the path to your .db file
db_file = '../data/chinook.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute an SQL query to retrieve the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Open the existing 'schema.txt' file in append mode
with open('schema.txt', 'a') as f:

    # Iterate through each table and append the content to the 'schema.txt' file
    for table in tables:
        table_name = table[0]
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        df = pd.read_sql_query(query, conn)
        
        # Append the table name and content to the 'schema.txt' file
        f.write(f"Table Name: {table_name}\n")
        f.write(df.to_string(index=False) + "\n")
        f.write("\n" + "="*40 + "\n")

print("Table content has been appended to 'schema.txt'.")


Table content has been appended to 'schema.txt'.


In [30]:
import sqlite3

# Replace 'your_database.db' with the path to your .db file
db_file = '../data/chinook.db'

# List of table names
table_names = [
    'albums',
    'artists',
    'customers',
    'employees',
    'genres',
    'invoices',
    'invoice_items',
    'media_types',
    'playlists',
    'playlist_track',
    'tracks'
]

# Open the existing 'schema.txt' file in append mode
with open('schema.txt', 'a') as f:

    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Dictionary to store linked columns for each table
    linked_columns = {}

    # Iterate through each table and identify linked columns
    for table_name in table_names:
        cursor.execute("PRAGMA foreign_key_list(" + table_name + ");")
        foreign_keys = cursor.fetchall()

        for foreign_key in foreign_keys:
            from_column = foreign_key[3]
            to_table = foreign_key[4]
            to_column = foreign_key[5]

            if table_name not in linked_columns:
                linked_columns[table_name] = []

            linked_columns[table_name].append((from_column, to_table, to_column))

    # Write the linked columns to the 'schema.txt' file
    for table, columns in linked_columns.items():
        f.write(f"Table: {table}\n")
        for column_info in columns:
            from_column, to_table, to_column = column_info
            f.write(f"  - {from_column} links to {to_table}.{to_column}\n")
        f.write("\n" + "="*40 + "\n")

# Notify the user
print("Linked columns have been appended to 'schema.txt'.")


Linked columns have been appended to 'schema.txt'.


**2. Find the ID of Aerosmith** 

Let's introduce here conditions by using the keyword `WHERE`. The goal is to retrieve only the rows that meet a given condition.

Here we will retrive here the column `artistId` from the table `artists` for the entries whose the name is `Aerosmith`.

In [31]:
query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = 'Aerosmith';
"""
cursor.execute(query1)
artist_id = cursor.fetchone()[0]

artist_id

3

**3. Find the IDs of the albums of Aerosmith**

Now that we have the `artistId` of Aerosmith we can use it in another query. We will retrieve the field `albumId` for the rows from album whose `artistId` is the one of Aerosmith.

In [9]:
query2 = f"""
    SELECT
        albumId
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
for row in cursor.execute(query2):
    albums_ids.append(str(row[0]))

print(albums_ids)

['5']


**4. Find the track names of the albums of Aerosmith**

Now that we have the ids of all the albums of Aerosmith we can look for the tracks that are in those albums. We will introduce here the `DISTINCT` keyword that will retrieve unique values only.


In [10]:
query3 = f"""
    SELECT
        DISTINCT name
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

for row in cursor.execute(query3):
    print(row[0])

Walk On Water
Love In An Elevator
Rag Doll
What It Takes
Dude (Looks Like A Lady)
Janie's Got A Gun
Cryin'
Amazing
Blind Man
Deuces Are Wild
The Other Side
Crazy
Eat The Rich
Angel
Livin' On The Edge


**5. Put that in a single query!**

Let's do everything we have done so far in a single query.

Before that it is important to understand the concept of `PRIMARY KEY` and `FOREIGN KEY`:

- **Primary Keys** are fields that are **unique** and cannot be null. It uniquely identifies each record in a table (in general this is the `id` of the item). Each table can have zero or one primary key (not more). This key can consist of one or multiple columns. 
- **Foreign Keys** are fields in one table that refer to the primary key of another table. One table can contain multiple foreign keys.

Let's take an example: 
* In our dataset the table `artists` has `artistId` as primary key. The table `albums` uses `artistId` as a foreign key to refer to the table `artists`. 
* The idea of joining table is based on that: we will use the `JOIN` keyword in order to put information from different tables together by using common keys. 
* The keyword `ON` will specify which keys will be used to join the tables.

In [8]:
query4 = f"""
    SELECT
        distinct(tracks.name)
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = 'Aerosmith';
"""

songs = set()
for row in cursor.execute(query4):
    print(row[0])
    

Walk On Water
Love In An Elevator
Rag Doll
What It Takes
Dude (Looks Like A Lady)
Janie's Got A Gun
Cryin'
Amazing
Blind Man
Deuces Are Wild
The Other Side
Crazy
Eat The Rich
Angel
Livin' On The Edge


**6. Get the most productive artists**

Let's count the number of albums by artist by using `GROUP BY`. We will select the artists and group all the albums they have released.

`COUNT` will count the number of albums and `AS` will give a variable name to the counter (`nb_albums`).

`ORDER BY` will rank the results according to the variable `nb_albums`. `DESC` gives the orientation of the rank (in that case: descending)

`LIMIT` will show only the first results of the ranking (i.e. the most productive artists).

In [7]:
query5 = """

    SELECT
        artists.Name,
        count(*) AS nb_albums
    FROM
        artists
        JOIN albums on albums.ArtistId = artists.ArtistId
    GROUP BY
        artists.ArtistId
    ORDER BY
        nb_albums DESC
    LIMIT 10;

"""
cursor.execute(query5)
cursor.fetchall()

[('Iron Maiden', 21),
 ('Led Zeppelin', 14),
 ('Deep Purple', 11),
 ('Metallica', 10),
 ('U2', 10),
 ('Ozzy Osbourne', 6),
 ('Pearl Jam', 5),
 ('Various Artists', 4),
 ('Faith No More', 4),
 ('Foo Fighters', 4)]

#### Your turn!

Based on your knowledge and some Google search try to create the following queries:

- Count the number of tracks for all the albums of AC/DC
- List all the artists doing 'Metal' (genre)
- List the top 10 of invoiced countries
- Calculate the total amount billed to each customer

In [34]:
#- Count the number of tracks for all the albums of AC/DC
query_1 = """
SELECT
    albums.AlbumId,
    albums.Title AS AlbumTitle,
    COUNT(tracks.TrackId) AS NumberOfTracks
FROM
    albums
JOIN artists ON artists.ArtistId = albums.ArtistId
JOIN tracks ON tracks.AlbumId = albums.AlbumId
WHERE artists.Name = 'AC/DC'
GROUP BY albums.AlbumId
ORDER BY albums.AlbumId;
"""
cursor.execute(query_1)
cursor.fetchall()

[(1, 'For Those About To Rock We Salute You', 10), (4, 'Let There Be Rock', 8)]

In [41]:
#- List all the artists doing 'Metal' (genre)
query_2 = """
SELECT DISTINCT
    tracks.Composer
FROM 
    tracks
JOIN genres ON tracks.GenreId = genres.GenreId
WHERE genres.Name = "Metal"

"""
cursor.execute(query_2)
cursor.fetchall()


[('Apocalyptica',),
 (None,),
 ('Tony Iommi, Bill Ward, Geezer Butler, Ozzy Osbourne',),
 ('Roy Z',),
 ('Adrian Smith',),
 ('Eddie Casillas/Roy Z',),
 ('Molaney/Morris/Roberts/Wainwright',),
 ('Harris/Tatler',),
 ('Black Sabbath',),
 ('Seger',),
 ('Danzig',),
 ('Cave',),
 ('Diamond/Shermann',),
 ('A.Bouchard/J.Bouchard/S.Pearlman',),
 ('Traditional',),
 ('Collins/Van Zandt',),
 ('Sully Erna',),
 ('Sully Erna; Tony Rombola',),
 ('Duff McKagan/Slash/W. Axl Rose',),
 ("Izzy Stradlin'/W. Axl Rose",),
 ('Billy/Del James/W. Axl Rose/West Arkeen',),
 ('Bob Dylan',),
 ('W. Axl Rose',),
 ("Izzy Stradlin'",),
 ('Slash/W. Axl Rose',),
 ('Duff McKagan',),
 ('Steve Harris',),
 ("Paul Di'Anno/Steve Harris",),
 ("David Murray/Paul Di'Anno/Steve Harris",),
 ('Adrian Smith/Bruce Dickinson',),
 ('Bruce Dickinson/Janick Gers',),
 ('Adrian Smith/Bruce Dickinson/Steve Harris',),
 ('Bruce Dickinson/Steve Harris',),
 ('Bruce Dickinson',),
 ('Smith/Dickinson',),
 ('Harris',),
 ('Dickinson',),
 ('Harris/Di Ann

In [43]:
#- List the top 10 of invoiced countries
query_3 = """
 
SELECT invoices.BillingCountry, COUNT(invoices.InvoiceId) AS NumberOfInvoices
FROM invoices
GROUP by invoices.BillingCountry
ORDER by NumberOfInvoices DESC
LIMIT 10;

"""
cursor.execute(query_3)
cursor.fetchall()



[('USA', 91),
 ('Canada', 56),
 ('France', 35),
 ('Brazil', 35),
 ('Germany', 28),
 ('United Kingdom', 21),
 ('Portugal', 14),
 ('Czech Republic', 14),
 ('India', 13),
 ('Sweden', 7)]

In [51]:
#- Calculate the total amount billed to each customer
query_4 = """
 
SELECT customers.FirstName || ' ' || customers.LastName AS CustomerName, SUM(invoices.Total) AS TotalInvoice
FROM invoices
JOIN customers ON invoices.CustomerId = customers.CustomerId
GROUP BY invoices.CustomerId
ORDER BY TotalInvoice DESC
LIMIT 10;

"""
cursor.execute(query_4)
cursor.fetchall()
# SELECT customers.FirstName + " " + customers.LastName AS name, SUM(invoices.Total) AS TotalInvoice


[('Helena Holý', 49.620000000000005),
 ('Richard Cunningham', 47.620000000000005),
 ('Luis Rojas', 46.62),
 ('Ladislav Kovács', 45.62),
 ("Hugh O'Reilly", 45.62),
 ('Julia Barnett', 43.620000000000005),
 ('Frank Ralston', 43.62),
 ('Fynn Zimmermann', 43.62),
 ('Astrid Gruber', 42.62),
 ('Victor Stevens', 42.62)]

In [ ]:
# Close the database connection
conn.close()

## Resources

- [Top 70 most important queries](https://bytescout.com/blog/20-important-sql-queries.html)
- [TablePlus](https://tableplus.com/): A nice editor for SQL queries
- [SQL Fundamentals ](https://app.datacamp.com/learn/skill-tracks/sql-fundamentals)(DataCamp Track, 5 courses)
- [SQL for Business Analysts ](https://app.datacamp.com/learn/skill-tracks/sql-for-business-analysts)(DataCamp Track, 5 courses)
- [Data Analyst in SQL ](https://app.datacamp.com/learn/career-tracks/data-analyst-in-sql)(DataCamp Track, 12 courses)
- [The 80 Top SQL Interview Questions and Answers for Beginners & Intermediate Practitioners](https://www.datacamp.com/blog/top-sql-interview-questions-and-answers-for-beginners-and-intermediate-practitioners)